# Compare the different approaches against each other using different metrics
We compare the following different similarity measures against each other:
- [B25](https://lucene.apache.org/core/9_7_0/core/org/apache/lucene/search/similarities/BM25Similarity.html)
- [TD-IDF](https://lucene.apache.org/core/9_7_0/core/org/apache/lucene/search/similarities/TFIDFSimilarity.html)
- [DFR](https://lucene.apache.org/core/9_7_0/core/org/apache/lucene/search/similarities/DFRSimilarity.html)
- [AxiomaticF2LOG](https://lucene.apache.org/core/9_7_0/core/org/apache/lucene/search/similarities/AxiomaticF2LOG.html)

For each of these we test queries with and without query expansion.
We evaluate by looking up similar positions to solution of the chess puzzle and see if the move gets correctly classified as winning.
We we evaluate the results using P@5, we opted for precision since it is not clear what the number of actual relevant documents is.

## Setup

In [100]:
# Import all the necessary modules.
from dataclasses import dataclass
from enum import Enum
from IPython.core.display import HTML
import chess
import chess.pgn
import lucene
import os
import pandas as pd
from pprint import pprint
from datetime import datetime

# from org.apache.lucene.analysis.standard import StandardAnalyzer
from java.nio.file import Paths
from org.apache.lucene.analysis.core import WhitespaceAnalyzer
from org.apache.lucene.index import DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.search.similarities import BM25Similarity, ClassicSimilarity
from org.apache.lucene.search.similarities import DFRSimilarity, BasicModelIn, AfterEffectB, NormalizationH2, AxiomaticF2LOG
from org.apache.lucene.store import NIOFSDirectory

In [71]:
# Set up the Java VM for Lucene.

try:
    print("Initializing VM")
    lucene.initVM(vmargs=['-Djava.awt.headless=true'])  # This needs to be run once to get it all going
except ValueError as e:
    print("VM already initialized")

Initializing VM
VM already initialized


In [72]:
# Logic to convert between the chess board and our representation.

val_map = {i: str(1 - (7 * i / 64)) for i in range(1, 9)}
def r_closure(board):
    t = board.turn
    board.turn = chess.WHITE
    w_none_capture_moves = [move for move in board.legal_moves if not board.is_capture(move)]
    w_r_closure = [board.piece_at(x.from_square).symbol() + chess.square_name(x.to_square) + '^' + val_map[chess.square_distance(x.from_square, x.to_square)] for x in w_none_capture_moves]

    board.turn = chess.BLACK
    b_none_capture_moves = [move for move in board.legal_moves if not board.is_capture(move)]
    b_r_closure = [board.piece_at(x.from_square).symbol() + chess.square_name(x.to_square) + '^' + val_map[chess.square_distance(x.from_square, x.to_square)] for x in b_none_capture_moves]

    board.turn = t
    return w_r_closure + b_r_closure

def a_closure(board):
    t = board.turn
    
    board.turn = chess.WHITE
    w_capture_moves = [move for move in board.legal_moves if board.is_capture(move) and not board.is_en_passant(move)]
    w_a_closure = [board.piece_at(x.from_square).symbol() + ">" + board.piece_at(x.to_square).symbol() + chess.square_name(x.to_square) for x in w_capture_moves]

    board.turn = chess.BLACK
    b_capture_moves = [move for move in board.legal_moves if board.is_capture(move) and not board.is_en_passant(move)]
    b_a_closure = [board.piece_at(x.from_square).symbol() + ">" + board.piece_at(x.to_square).symbol() + chess.square_name(x.to_square) for x in b_capture_moves]

    board.turn = t
    return w_a_closure + b_a_closure

def d_closure(board):
    d_closure = []

    for square, piece in board.piece_map().items():
        if piece.piece_type != chess.KING: # Defending a king doesn't really make sense
            d_closure += [board.piece_at(x).symbol() + '<' + piece.symbol() + chess.square_name(square) for x in board.attackers(piece.color, square)]

    return d_closure

def p_closure(board):
    return [p.symbol()+chess.square_name(s) for s,p in board.piece_map().items()]

def encode_board_position_expansion(board):
    return  " ".join(r_closure(board)) + " " + " ".join(p_closure(board)) +" "+ " ".join(a_closure(board)) +" "+ " ".join(d_closure(board))

def encode_board_position_no_expansion(board):
    return  " " + " ".join(p_closure(board)) +" "+ " ".join(a_closure(board)) +" "+ " ".join(d_closure(board))

def decode_piece_position(pos):
    board = chess.Board()
    board.set_piece_map({chess.parse_square(term[1:]): chess.Piece.from_symbol(term[0]) for term in pos.strip().split(" ")})
    return board


In [73]:
# Load the puzzles to evaluate on from the CSV file.

df = pd.read_csv('../csvs/sorted_df_output.csv')
print(df.shape)

def get_puzzle_board(index):
    fen = df.iloc[index]['FEN']
    board = chess.Board(fen)
    board.push(chess.Move.from_uci(df.iloc[index]['Moves'].split(" ")[0]))
    return board

def get_puzzle_solution(index):
    return df.iloc[index]['Moves'].split(" ")[1] # Only the first move

def get_position_before_puzzle(index):
    fen = df.iloc[index]['FEN']
    board = chess.Board(fen)
    # board.push(chess.Move.from_uci(df.iloc[index]['Moves'].split(" ")[0]))
    return board

def get_prep_move(index):
    return df.iloc[index]['Moves'].split(" ")[0]

(1000, 10)


In [74]:
PATH = "../games/"

def run_query(searcher, analyzer, command, bound=10):
    """ Runs the actuall lucene query and collects the output. """
    escaped_command = command.replace("/", "\\/")
    query = QueryParser("contents", analyzer).parse(escaped_command)
    scoreDocs = searcher.search(query, bound).scoreDocs
    
    seen_games = set()
    output = []
    for scoreDoc in scoreDocs:
        doc = searcher.doc(scoreDoc.doc)
        link = f"https://lichess.org/{doc.get('game').split('.')[0]}#{int(doc.get('move'))}"
        
        # Skip any double matches for the games.
        if doc.get("game") in seen_games:
            continue
        
        file = open(os.path.join(PATH, doc.get("game")))
        file_content = str(file.read())
        content = file_content.split("\n")[int(doc.get("move"))-10].split("|")[0]
        header = file_content.split("\n")[0]
        file.close()
        
        output.append([doc.get('game'), doc.get('move'), link, scoreDoc.score, content, header])
        seen_games.add(doc.get("game"))
    
    return output

def score_lookup(board, bound, query_expansion=True):
    """ Wrapper for the lucene query to make it all function. """
    # output = []
    # directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
    # searcher = IndexSearcher(DirectoryReader.open(directory))
    # searcher.setSimilarity(BM25Similarity())
    # analyzer = WhitespaceAnalyzer()
    # output = run_query(searcher, analyzer, command=encode_board_position(board), bound=bound)
    # del searcher  
    # return output 
    if query_expansion:
        return run_query(searcher, analyzer, command=encode_board_position_expansion(board), bound=bound)
    else:
        return run_query(searcher, analyzer, command=encode_board_position_no_expansion(board), bound=bound)


In [75]:
# The experiment logic.
class Outcome(Enum):
    WHITE_WON = 1
    BLACK_WON = -1
    DRAW = 0

# turn = 1 white and 0 black
def evaluate_move(turn, move, board, bound, top=10, query_expansion=True):
    output = score_lookup(board, bound, query_expansion)
    if len(output) < 5:
        raise ValueError(f"Not enough results found for {board} with bound={bound}.")
    
    # Sort the output by score and get the top_k
    output.sort(key=lambda x: x[3], reverse=True)
    top_k = output[:top]
    
    filtered_top_k = []
    for result in top_k:
        outcome = Outcome.WHITE_WON if "1-0" in result[5] else Outcome.BLACK_WON if "0-1" in result[5] else Outcome.DRAW
        filtered_top_k.append([result[3], outcome ,result[2], result[4]])
    
    # Calculate the score (unfiltered for now)
    if turn == 1:
        # white_wins = sum([1 for x in filtered_top_k if x[1] == Outcome.WHITE_WON])
        # score = white_wins / len(filtered_top_k)
        
        p_1 = sum([1 for x in filtered_top_k[:1] if x[1] == Outcome.WHITE_WON])
        p_5 = sum([1 for x in filtered_top_k[:5] if x[1] == Outcome.WHITE_WON])
        p_10 = sum([1 for x in filtered_top_k[:10] if x[1] == Outcome.WHITE_WON])
        
        # print(f"w: {p_1}, {p_5}, {p_10}")
    else:
        # black_wins = sum([1 for x in filtered_top_k if x[1] == Outcome.BLACK_WON])
        # score = black_wins / len(filtered_top_k)
        
        p_1 = sum([1 for x in filtered_top_k[:1] if x[1] == Outcome.BLACK_WON])
        p_5 = sum([1 for x in filtered_top_k[:5] if x[1] == Outcome.BLACK_WON])
        p_10 = sum([1 for x in filtered_top_k[:10] if x[1] == Outcome.BLACK_WON])
        
        #print(f"b: {p_1}, {p_5}, {p_10}")
    # TODO: Also want the precision for the top_k with the correct color already.
    
    return [[p_1, p_5, p_10], [p_1 > 0.5, p_5 > 2.5, p_10 >5], filtered_top_k]


def evaluate_position(turn, board, print_progress=False, bound=50, top=5, query_expansion=True):
    output = []
    
    for move in board.legal_moves:
        if print_progress: print(f"{move}, ", end="")
        board.push(move)
        output.append(evaluate_move(turn, move, board, bound, top, query_expansion))
        board.pop()
    
    return output

## BM25

### With query expansion

In [76]:
b25_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(BM25Similarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    b25_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(b25_output)

del searcher

0,10,20,30,40,50,60,70,80,90,/nRan for:0:02:55.175113
[[[1, 3, 5], [True, True, False], [[42.50636672973633, <Outcome.BLACK_WON: -1>, 'https://lichess.org/3lEsQvLU#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 nf6 pa6 Pf5 pd5 bc5 pb5 Pe4 Qd4 Ph3 Bd3 Nc3 Pa3 Pg2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [41.92009735107422, <Outcome.WHITE_WON: 1>, 'https://lichess.org/NtROTkkp#42', 'kg8 rc8 rb8 ph7 pg7 pf7 qd7 nf6 pb6 pd5 bc5 Pb5 pa5 Pe4 Qd4 Pa4 Pg3 Nc3 Ph2 Kg2 Pf2 Bb2 Rd1 Rc1 '], [41.68668746948242, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [41.56430435180664, <Outcome.BLACK_WON: -1>, 'https://lichess.org/VoxEixrZ#32', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 bb7 pe6 pa6 pd5 bc5 pb5 Pg4 Pf4 Pe4 Qd4 Nc3 Pa3 Pg2 Pc2 Pb2 Ba2 Kg1 Rf1 Rd1 Bc1 '], [41.562225341796875, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2

In [77]:
# analyze the results
p_1values = [x[0][0]/1 for x in b25_output]
p_5values = [x[0][1]/5 for x in b25_output]
p_10values = [x[0][2]/10 for x in b25_output]

s_1values = [x[1][0] for x in b25_output]
s_5values = [x[1][1] for x in b25_output]
s_10values = [x[1][2] for x in b25_output]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

AVG: p@1: 0.55, p@5: 0.5540000000000002, p@10: 0.512
SCORE: @1: 55, @5: 63, @10: 39


### Without query expansion

In [78]:
b25_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(BM25Similarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    b25_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(b25_output_nqe)

del searcher

0,10,20,30,40,50,60,70,80,90,/nRan for:0:01:05.168217
[[[0, 3, 5], [False, True, False], [[39.3881950378418, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2 Pb2 Kh1 Rf1 Ra1 '], [38.62403869628906, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Kb4aJX11#30', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 pb7 pa7 nf6 be6 Bg5 bc5 Pf4 pc4 Qe3 Nc3 Ph2 Pg2 Be2 Pc2 Pb2 Pa2 Kg1 Rf1 Ra1 '], [38.62248992919922, <Outcome.BLACK_WON: -1>, 'https://lichess.org/cMwfAYpA#34', 'kg8 rf8 qd8 ph7 pg7 pf7 bb7 nf6 pe6 pb6 pa6 pd5 bc5 Nf4 Pe4 Pc4 Pa4 Pf3 Qe3 Nc3 Ph2 Pg2 Bd2 Pc2 Kg1 Rf1 Rd1 '], [38.40948486328125, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [38.28981018066406, <Outcome.BLACK_WON: -1>, 'https://lichess.org/jX6sJGdo#30', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pa7 pe6 nf5 pd5 bc5 Pf4 Qf3 Nc3 Pa3 Ph2 Pg2 Bd2 

In [79]:
# analyze the results
p_1values = [x[0][0]/1 for x in b25_output_nqe]
p_5values = [x[0][1]/5 for x in b25_output_nqe]
p_10values = [x[0][2]/10 for x in b25_output_nqe]

s_1values = [x[1][0] for x in b25_output_nqe]
s_5values = [x[1][1] for x in b25_output_nqe]
s_10values = [x[1][2] for x in b25_output_nqe]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

AVG: p@1: 0.58, p@5: 0.5559999999999998, p@10: 0.5190000000000001
SCORE: @1: 58, @5: 55, @10: 41


### Summary of results

In [80]:
print(b25_output)
print(b25_output_nqe)

[[[1, 3, 5], [True, True, False], [[42.50636672973633, <Outcome.BLACK_WON: -1>, 'https://lichess.org/3lEsQvLU#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 nf6 pa6 Pf5 pd5 bc5 pb5 Pe4 Qd4 Ph3 Bd3 Nc3 Pa3 Pg2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [41.92009735107422, <Outcome.WHITE_WON: 1>, 'https://lichess.org/NtROTkkp#42', 'kg8 rc8 rb8 ph7 pg7 pf7 qd7 nf6 pb6 pd5 bc5 Pb5 pa5 Pe4 Qd4 Pa4 Pg3 Nc3 Ph2 Kg2 Pf2 Bb2 Rd1 Rc1 '], [41.68668746948242, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [41.56430435180664, <Outcome.BLACK_WON: -1>, 'https://lichess.org/VoxEixrZ#32', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 bb7 pe6 pa6 pd5 bc5 pb5 Pg4 Pf4 Pe4 Qd4 Nc3 Pa3 Pg2 Pc2 Pb2 Ba2 Kg1 Rf1 Rd1 Bc1 '], [41.562225341796875, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2 Pb2 Kh1 Rf1 Ra1 '], [40.995582580566406, <Outcome.BLA

### Test move before to check that the we are not just biased to posivite results


#### With query expansion

In [81]:
prep_b25_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(BM25Similarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    prep_b25_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"/nRan for:{end - start}")
print(prep_b25_output)

del searcher

0,10,20,30,40,50,60,70,80,90,/nRan for:0:03:14.878176
[[[0, 2, 5], [False, False, False], [[47.31442642211914, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [45.38957977294922, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [45.173824310302734, <Outcome.WHITE_WON: 1>, 'https://lichess.org/bt2YHlEj#38', 'kg8 rd8 rc8 pg7 pf7 be7 qd7 pg6 nf6 pe6 na6 pd5 pb5 pa5 Pe4 Pg3 Be3 Nc3 Pa3 Ph2 Bg2 Pf2 Qc2 Pb2 Kg1 Re1 Rd1 '], [45.013946533203125, <Outcome.DRAW: 0>, 'https://lichess.org/LyWJ3OGS#30', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 rc6 pa6 Bg5 pd5 Pd4 Qd3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Re1 Rd1 '], [44.77211380004883, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#31', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 rc6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph

In [82]:
# analyze the results: (NOTE: here we expect the move to be bad!)
p_1values = [(1-x[0][0])/1 for x in prep_b25_output]
p_5values = [(5-x[0][1])/5 for x in prep_b25_output]
p_10values = [(10-x[0][2])/10 for x in prep_b25_output]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

AVG: p@1: 0.51, p@5: 0.5700000000000001, p@10: 0.5440000000000002
SCORE: @1: 51, @5: 61, @10: 48


#### Without query expansion

In [83]:
prep_b25_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(BM25Similarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    prep_b25_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_b25_output_nqe)

del searcher

0,10,20,30,40,50,60,70,80,90,
Ran for:0:01:07.788538
[[[1, 2, 5], [True, False, False], [[43.49882507324219, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [43.264305114746094, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [41.839847564697266, <Outcome.WHITE_WON: 1>, 'https://lichess.org/xq0l0EoT#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Ph3 Qf3 Nc3 Pa3 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rc1 '], [41.644378662109375, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#25', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pd3 Nc3 Ph2 Pg2 Pf2 Qe2 Pc2 Pb2 Pa2 Kg1 Rf1 Rd1 '], [40.736881256103516, <Outcome.BLACK_WON: -1>, 'https://lichess.org/MC4lkIzo#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 be7 nf6 pa6 Bg5 Qe5 pd5 Pe4 bh3 Pg3 Nc3 Ph2 Pf2 

In [84]:
# analyze the results: (NOTE: here we expect the move to be bad!)
p_1values = [(1-x[0][0])/1 for x in prep_b25_output_nqe]
p_5values = [(5-x[0][1])/5 for x in prep_b25_output_nqe]
p_10values = [(10-x[0][2])/10 for x in prep_b25_output_nqe]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

AVG: p@1: 0.63, p@5: 0.5820000000000001, p@10: 0.5529999999999998
SCORE: @1: 63, @5: 63, @10: 56


## Results BM25

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.55, p@5: 0.5540000000000002, p@10: 0.512 <br> SCORE: @1: 55, @5: 63, @10: 39         | AVG: p@1: 0.58, p@5: 0.5559999999999998, p@10: 0.5190000000000001 <br> SCORE: @1: 58, @5: 55, @10: 41         |
| Pre puzzle move (bad move)  | AVG: p@1: 0.51, p@5: 0.5700000000000001, p@10: 0.5440000000000002 <br> SCORE: @1: 51, @5: 61, @10: 48         |  AVG: p@1: 0.63, p@5: 0.5820000000000001, p@10: 0.5529999999999998 <br> SCORE: @1: 63, @5: 63, @10: 56        |



# Classic Model (TFIDFSimilarity)

## Puzzle solution (good move)

In [87]:
classic_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(ClassicSimilarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    classic_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(classic_output)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in classic_output]
p_5values = [x[0][1]/5 for x in classic_output]
p_10values = [x[0][2]/10 for x in classic_output]

s_1values = [x[1][0] for x in classic_output]
s_5values = [x[1][1] for x in classic_output]
s_10values = [x[1][2] for x in classic_output]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:18:13.222382
[[[1, 4, 6], [True, True, True], [[18.825048446655273, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [18.792829513549805, <Outcome.BLACK_WON: -1>, 'https://lichess.org/3lEsQvLU#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 nf6 pa6 Pf5 pd5 bc5 pb5 Pe4 Qd4 Ph3 Bd3 Nc3 Pa3 Pg2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [18.73036003112793, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2 Pb2 Kh1 Rf1 Ra1 '], [18.46516990661621, <Outcome.BLACK_WON: -1>, 'https://lichess.org/VoxEixrZ#32', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 bb7 pe6 pa6 pd5 bc5 pb5 Pg4 Pf4 Pe4 Qd4 Nc3 Pa3 Pg2 Pc2 Pb2 Ba2 Kg1 Rf1 Rd1 Bc1 '], [18.384658813476562, <Outcome.BLACK_WON: -1>, 'https://lichess.org/KcnejAfR#25', 'kg8 re8 rb8 ph7 pg7 pf7 qd7 pb7 nf6 pe6 bd6 pa6 pd5 Pe4 Qd4 Nf3 Nc3 Pa3 Ph2 Pg

In [88]:
classic_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(ClassicSimilarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    classic_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(classic_output_nqe)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in classic_output_nqe]
p_5values = [x[0][1]/5 for x in classic_output_nqe]
p_10values = [x[0][2]/10 for x in classic_output_nqe]

s_1values = [x[1][0] for x in classic_output_nqe]
s_5values = [x[1][1] for x in classic_output_nqe]
s_10values = [x[1][2] for x in classic_output_nqe]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:10:58.914566
[[[0, 2, 6], [False, False, True], [[17.74761199951172, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2 Pb2 Kh1 Rf1 Ra1 '], [17.488327026367188, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [17.182598114013672, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [17.052749633789062, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Kb4aJX11#30', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 pb7 pa7 nf6 be6 Bg5 bc5 Pf4 pc4 Qe3 Nc3 Ph2 Pg2 Be2 Pc2 Pb2 Pa2 Kg1 Rf1 Ra1 '], [17.050756454467773, <Outcome.WHITE_WON: 1>, 'https://lichess.org/nTVI4R6W#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 Nc6 pa6 pd5 pb5 Pe4 Bg3 Pf3 Nc3 Ph2 P

## Pre puzzle move (bad move)

In [90]:
prep_classic_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(ClassicSimilarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    prep_classic_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_classic_output)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_classic_output]
p_5values = [(5-x[0][1])/5 for x in prep_classic_output]
p_10values = [(10-x[0][2])/10 for x in prep_classic_output]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:19:38.322375
[[[0, 2, 4], [False, False, False], [[21.040590286254883, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [20.285459518432617, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [20.05738067626953, <Outcome.WHITE_WON: 1>, 'https://lichess.org/xq0l0EoT#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 pa7 nf6 pe6 Bg5 pd5 Ph3 Nc3 Pa3 Pg2 Pf2 Qe2 Pc2 Pb2 Kg1 Rf1 Rc1 '], [19.853567123413086, <Outcome.DRAW: 0>, 'https://lichess.org/LyWJ3OGS#30', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 rc6 pa6 Bg5 pd5 Pd4 Qd3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Re1 Rd1 '], [19.77977752685547, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#31', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 rc6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf

In [92]:
prep_classic_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(ClassicSimilarity())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    prep_classic_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_classic_output_nqe)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_classic_output_nqe]
p_5values = [(5-x[0][1])/5 for x in prep_classic_output_nqe]
p_10values = [(10-x[0][2])/10 for x in prep_classic_output_nqe]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:11:10.303872
[[[1, 3, 4], [True, True, False], [[19.511810302734375, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [19.264745712280273, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [18.891721725463867, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#25', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pd3 Nc3 Ph2 Pg2 Pf2 Qe2 Pc2 Pb2 Pa2 Kg1 Rf1 Rd1 '], [18.89011001586914, <Outcome.WHITE_WON: 1>, 'https://lichess.org/xq0l0EoT#26', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 pa7 nf6 pe6 Bg5 pd5 Ph3 Nc3 Pa3 Pg2 Pf2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [18.263662338256836, <Outcome.WHITE_WON: 1>, 'https://lichess.org/nTVI4R6W#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 Nc6 pa6 pd5 pb5 Pe4 Bg3 Pf3 Nc3 Ph2 Pg

# Results TFIDF

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.56, p@5: 0.5399999999999999, p@10: 0.4970000000000001 <br> SCORE: @1: 56, @5: 56, @10: 36 | AVG: p@1: 0.53, p@5: 0.5679999999999998, p@10: 0.529 <br> SCORE: @1: 53, @5: 60, @10: 44 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.51, p@5: 0.5459999999999999, p@10: 0.5619999999999998 <br> SCORE: @1: 51, @5: 52, @10: 47 | AVG: p@1: 0.58, p@5: 0.5960000000000001, p@10: 0.5559999999999999 <br> SCORE: @1: 58, @5: 67, @10: 54 |

# Divergence from randomness(DFR)

## Puzzle solution (good move)

In [94]:
dfr_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(DFRSimilarity(BasicModelIn(), AfterEffectB(), NormalizationH2()))
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    dfr_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(dfr_output)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in dfr_output]
p_5values = [x[0][1]/5 for x in dfr_output]
p_10values = [x[0][2]/10 for x in dfr_output]

s_1values = [x[1][0] for x in dfr_output]
s_5values = [x[1][1] for x in dfr_output]
s_10values = [x[1][2] for x in dfr_output]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:03:01.012313
[[[1, 4, 4], [True, True, False], [[68.96511840820312, <Outcome.BLACK_WON: -1>, 'https://lichess.org/3lEsQvLU#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 nf6 pa6 Pf5 pd5 bc5 pb5 Pe4 Qd4 Ph3 Bd3 Nc3 Pa3 Pg2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [68.45819854736328, <Outcome.WHITE_WON: 1>, 'https://lichess.org/NtROTkkp#42', 'kg8 rc8 rb8 ph7 pg7 pf7 qd7 nf6 pb6 pd5 bc5 Pb5 pa5 Pe4 Qd4 Pa4 Pg3 Nc3 Ph2 Kg2 Pf2 Bb2 Rd1 Rc1 '], [68.10696411132812, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [67.77072143554688, <Outcome.BLACK_WON: -1>, 'https://lichess.org/VoxEixrZ#32', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 bb7 pe6 pa6 pd5 bc5 pb5 Pg4 Pf4 Pe4 Qd4 Nc3 Pa3 Pg2 Pc2 Pb2 Ba2 Kg1 Rf1 Rd1 Bc1 '], [67.35856628417969, <Outcome.BLACK_WON: -1>, 'https://lichess.org/KcnejAfR#25', 'kg8 re8 rb8 ph7 pg7 pf7 qd7 pb7 nf6 pe6 bd6 pa6 pd5 Pe4 Qd4 Nf3 Nc3 Pa3 Ph2 Pg2 Pf2 

In [95]:
dfr_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(DFRSimilarity(BasicModelIn(), AfterEffectB(), NormalizationH2()))
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    dfr_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(dfr_output_nqe)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in dfr_output_nqe]
p_5values = [x[0][1]/5 for x in dfr_output_nqe]
p_10values = [x[0][2]/10 for x in dfr_output_nqe]

s_1values = [x[1][0] for x in dfr_output_nqe]
s_5values = [x[1][1] for x in dfr_output_nqe]
s_10values = [x[1][2] for x in dfr_output_nqe]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:01:06.127190
[[[0, 3, 5], [False, True, False], [[63.486385345458984, <Outcome.WHITE_WON: 1>, 'https://lichess.org/v85b1iMi#32', 'kg8 rf8 rc8 ph7 pg7 pf7 qd7 pb7 pe6 pa6 nf5 Pe5 pd5 bc5 Qg4 Pf4 Nc3 Pa3 Ph2 Pg2 Bd2 Pb2 Kh1 Rf1 Ra1 '], [63.085716247558594, <Outcome.BLACK_WON: -1>, 'https://lichess.org/cMwfAYpA#34', 'kg8 rf8 qd8 ph7 pg7 pf7 bb7 nf6 pe6 pb6 pa6 pd5 bc5 Nf4 Pe4 Pc4 Pa4 Pf3 Qe3 Nc3 Ph2 Pg2 Bd2 Pc2 Kg1 Rf1 Rd1 '], [62.84066390991211, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [62.357826232910156, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Kb4aJX11#30', 'kg8 rf8 ra8 ph7 pg7 pf7 qc7 pb7 pa7 nf6 be6 Bg5 bc5 Pf4 pc4 Qe3 Nc3 Ph2 Pg2 Be2 Pc2 Pb2 Pa2 Kg1 Rf1 Ra1 '], [62.211463928222656, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 

## Pre puzzle move (bad move)

In [96]:
prep_dfr_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(DFRSimilarity(BasicModelIn(), AfterEffectB(), NormalizationH2()))
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    prep_dfr_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_dfr_output)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_dfr_output]
p_5values = [(5-x[0][1])/5 for x in prep_dfr_output]
p_10values = [(10-x[0][2])/10 for x in prep_dfr_output]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:03:16.260762
[[[0, 2, 5], [False, False, False], [[77.19570922851562, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [74.22887420654297, <Outcome.WHITE_WON: 1>, 'https://lichess.org/bt2YHlEj#38', 'kg8 rd8 rc8 pg7 pf7 be7 qd7 pg6 nf6 pe6 na6 pd5 pb5 pa5 Pe4 Pg3 Be3 Nc3 Pa3 Ph2 Bg2 Pf2 Qc2 Pb2 Kg1 Re1 Rd1 '], [74.14637756347656, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [73.31097412109375, <Outcome.DRAW: 0>, 'https://lichess.org/LyWJ3OGS#30', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 rc6 pa6 Bg5 pd5 Pd4 Qd3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Re1 Rd1 '], [72.9210433959961, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#31', 'kg8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 rc6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg

In [97]:
prep_dfr_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(DFRSimilarity(BasicModelIn(), AfterEffectB(), NormalizationH2()))
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    prep_dfr_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_dfr_output_nqe)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_dfr_output_nqe]
p_5values = [(5-x[0][1])/5 for x in prep_dfr_output_nqe]
p_10values = [(10-x[0][2])/10 for x in prep_dfr_output_nqe]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:01:10.107046
[[[1, 3, 5], [True, True, False], [[71.08780670166016, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [70.6654281616211, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [68.19325256347656, <Outcome.WHITE_WON: 1>, 'https://lichess.org/xq0l0EoT#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Ph3 Qf3 Nc3 Pa3 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rc1 '], [67.88632202148438, <Outcome.BLACK_WON: -1>, 'https://lichess.org/Xya5vvTA#25', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pd3 Nc3 Ph2 Pg2 Pf2 Qe2 Pc2 Pb2 Pa2 Kg1 Rf1 Rd1 '], [66.5125732421875, <Outcome.WHITE_WON: 1>, 'https://lichess.org/bt2YHlEj#38', 'kg8 rd8 rc8 pg7 pf7 be7 qd7 pg6 nf6 pe6 na6 pd5 pb5 pa5 Pe4 Pg3 Be3 Nc3 Pa3 Ph2 Bg2 

# Results DFR

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.56, p@5: 0.556, p@10: 0.526 <br> SCORE: @1: 56, @5: 62, @10: 41 | AVG: p@1: 0.57, p@5: 0.5419999999999998, p@10: 0.5220000000000001 <br> SCORE: @1: 57, @5: 54, @10: 41 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.52, p@5: 0.588, p@10: 0.552 <br> SCORE: @1: 52, @5: 66, @10: 51 | AVG: p@1: 0.64, p@5: 0.5660000000000001, p@10: 0.5519999999999999 <br> SCORE: @1: 64, @5: 58, @10: 55 |

# AxiomaticF2LOG

## Puzzle solution (good move)

In [101]:
ax_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(AxiomaticF2LOG())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    ax_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(ax_output)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in ax_output]
p_5values = [x[0][1]/5 for x in ax_output]
p_10values = [x[0][2]/10 for x in ax_output]

s_1values = [x[1][0] for x in ax_output]
s_5values = [x[1][1] for x in ax_output]
s_10values = [x[1][2] for x in ax_output]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:02:07.963700
[[[0, 2, 5], [False, False, False], [[67.64325714111328, <Outcome.WHITE_WON: 1>, 'https://lichess.org/i9YqMVj8#40', 'kg8 rf8 rc8 pg7 nf7 qe7 pb7 pa7 ph6 nf6 pe6 pd5 bc5 Pf4 Pe4 pc4 Ph3 Be3 Nc3 Pa3 Pg2 Ne2 Qd2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [67.47390747070312, <Outcome.WHITE_WON: 1>, 'https://lichess.org/NtROTkkp#42', 'kg8 rc8 rb8 ph7 pg7 pf7 qd7 nf6 pb6 pd5 bc5 Pb5 pa5 Pe4 Qd4 Pa4 Pg3 Nc3 Ph2 Kg2 Pf2 Bb2 Rd1 Rc1 '], [67.35867309570312, <Outcome.BLACK_WON: -1>, 'https://lichess.org/3lEsQvLU#34', 'kg8 rf8 qd8 rc8 ph7 pg7 pf7 nf6 pa6 Pf5 pd5 bc5 pb5 Pe4 Qd4 Ph3 Bd3 Nc3 Pa3 Pg2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [67.08829498291016, <Outcome.DRAW: 0>, 'https://lichess.org/fk72IrgR#24', 'kg8 rf8 qd8 bc8 ra8 ph7 pg7 pf7 be7 nf6 pe6 pa6 Bg5 pd5 pb5 Pf4 Pe4 Qd4 Nc3 Bb3 Ph2 Pg2 Pc2 Pb2 Pa2 Kg1 Rf1 Rd1 '], [66.92781066894531, <Outcome.BLACK_WON: -1>, 'https://lichess.org/KcnejAfR#25', 'kg8 re8 rb8 ph7 pg7 pf7 qd7 pb7 nf6 pe6 bd6 pa6 pd5 Pe4 Qd4 Nf3 Nc3 Pa3 Ph2 P

In [103]:
ax_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(AxiomaticF2LOG())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_puzzle_board(i)
    solution = chess.Move.from_uci(get_puzzle_solution(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    ax_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(ax_output_nqe)

del searcher

# analyze the results
p_1values = [x[0][0]/1 for x in ax_output_nqe]
p_5values = [x[0][1]/5 for x in ax_output_nqe]
p_10values = [x[0][2]/10 for x in ax_output_nqe]

s_1values = [x[1][0] for x in ax_output_nqe]
s_5values = [x[1][1] for x in ax_output_nqe]
s_10values = [x[1][2] for x in ax_output_nqe]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:00:37.465194
[[[0, 1, 4], [False, False, False], [[62.70953369140625, <Outcome.WHITE_WON: 1>, 'https://lichess.org/i9YqMVj8#40', 'kg8 rf8 rc8 pg7 nf7 qe7 pb7 pa7 ph6 nf6 pe6 pd5 bc5 Pf4 Pe4 pc4 Ph3 Be3 Nc3 Pa3 Pg2 Ne2 Qd2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [62.157066345214844, <Outcome.BLACK_WON: -1>, 'https://lichess.org/cMwfAYpA#34', 'kg8 rf8 qd8 ph7 pg7 pf7 bb7 nf6 pe6 pb6 pa6 pd5 bc5 Nf4 Pe4 Pc4 Pa4 Pf3 Qe3 Nc3 Ph2 Pg2 Bd2 Pc2 Kg1 Rf1 Rd1 '], [61.82324981689453, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [61.2585334777832, <Outcome.DRAW: 0>, 'https://lichess.org/fk72IrgR#24', 'kg8 rf8 qd8 bc8 ra8 ph7 pg7 pf7 be7 nf6 pe6 pa6 Bg5 pd5 pb5 Pf4 Pe4 Qd4 Nc3 Bb3 Ph2 Pg2 Pc2 Pb2 Pa2 Kg1 Rf1 Rd1 '], [61.2230339050293, <Outcome.WHITE_WON: 1>, 'https://lichess.org/nTVI4R6W#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 Nc6 pa6 pd5 pb5 Pe4 Bg3

## Pre puzzle move (bad move)

In [104]:
prep_ax_output = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(AxiomaticF2LOG())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=True)
    puzzle.pop()

    prep_ax_output.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_ax_output)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_ax_output]
p_5values = [(5-x[0][1])/5 for x in prep_ax_output]
p_10values = [(10-x[0][2])/10 for x in prep_ax_output]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:02:25.398841
[[[0, 3, 4], [False, True, False], [[74.97785186767578, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Re1 '], [73.8088607788086, <Outcome.WHITE_WON: 1>, 'https://lichess.org/bt2YHlEj#38', 'kg8 rd8 rc8 pg7 pf7 be7 qd7 pg6 nf6 pe6 na6 pd5 pb5 pa5 Pe4 Pg3 Be3 Nc3 Pa3 Ph2 Bg2 Pf2 Qc2 Pb2 Kg1 Re1 Rd1 '], [73.65172576904297, <Outcome.WHITE_WON: 1>, 'https://lichess.org/DrFwF87Q#30', 'kg8 rf8 qd8 rc8 ba8 ph7 pg7 pf7 be7 nf6 pe6 nb6 pa6 Bg5 pd5 pb5 Na5 Pf4 Pe4 Pb4 Bd3 Nc3 Pa3 Ph2 Pg2 Qd2 Pc2 Kg1 Rf1 Re1 '], [73.05828857421875, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [72.2064208984375, <Outcome.BLACK_WON: -1>, 'https://lichess.org/NuUF0yR6#36', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 bb7 nf6 pa6 Pf5 pe5 pd5 pb5

In [105]:
prep_ax_output_nqe = []

directory = NIOFSDirectory(Paths.get("/Users/tobiaswilfert/Documents/University/IR/chess-db/games.index"))
searcher = IndexSearcher(DirectoryReader.open(directory))
searcher.setSimilarity(AxiomaticF2LOG())
analyzer = WhitespaceAnalyzer()

start = datetime.now()
for i in range(0,100):
    puzzle = get_position_before_puzzle(i)
    solution = chess.Move.from_uci(get_prep_move(i))
    # display(HTML(chess.svg.board(board=puzzle, lastmove=solution, size=200)))

    puzzle.push(solution)
    temp = evaluate_move(not puzzle.turn, solution, puzzle, bound=100, top=10, query_expansion=False)
    puzzle.pop()

    prep_ax_output_nqe.append(temp)
    if i % 10 == 0:
        print(i, end=",")

end = datetime.now()
print(f"\nRan for:{end - start}")
print(prep_ax_output_nqe)

del searcher

# analyze the results
p_1values = [(1-x[0][0])/1 for x in prep_ax_output_nqe]
p_5values = [(5-x[0][1])/5 for x in prep_ax_output_nqe]
p_10values = [(10-x[0][2])/10 for x in prep_ax_output_nqe]

s_1values = [x > 0.5 for x in p_1values]
s_5values = [x > 0.5 for x in p_5values]
s_10values = [x > 0.5 for x in p_10values]

print(f"AVG: p@1: {sum(p_1values)/len(p_1values)}, p@5: {sum(p_5values)/len(p_5values)}, p@10: {sum(p_10values)/len(p_10values)}")
print(f"SCORE: @1: {sum(s_1values)}, @5: {sum(s_5values)}, @10: {sum(s_10values)}")

0,10,20,30,40,50,60,70,80,90,
Ran for:0:00:39.823896
[[[1, 4, 6], [True, True, True], [[70.01496887207031, <Outcome.WHITE_WON: 1>, 'https://lichess.org/YN8Ah8T6#27', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 nf6 pe6 pa6 Bg5 pd5 pb5 Pd4 Qf3 Nc3 Pa3 Ph2 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rd1 '], [68.55973815917969, <Outcome.BLACK_WON: -1>, 'https://lichess.org/hsII84BL#28', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Pf4 Pd4 Pa4 Ph3 Nc3 Pg2 Qe2 Pc2 Pb2 Kg1 Rf1 Ra1 '], [66.73119354248047, <Outcome.WHITE_WON: 1>, 'https://lichess.org/DrFwF87Q#30', 'kg8 rf8 qd8 rc8 ba8 ph7 pg7 pf7 be7 nf6 pe6 nb6 pa6 Bg5 pd5 pb5 Na5 Pf4 Pe4 Pb4 Bd3 Nc3 Pa3 Ph2 Pg2 Qd2 Pc2 Kg1 Rf1 Re1 '], [66.30243682861328, <Outcome.WHITE_WON: 1>, 'https://lichess.org/xq0l0EoT#29', 'kg8 rf8 rc8 ph7 pg7 pf7 be7 qd7 pb7 nf6 pe6 pa6 Bg5 pd5 Ph3 Qf3 Nc3 Pa3 Pg2 Pf2 Pc2 Pb2 Kg1 Rf1 Rc1 '], [66.03960418701172, <Outcome.WHITE_WON: 1>, 'https://lichess.org/bt2YHlEj#38', 'kg8 rd8 rc8 pg7 pf7 be7 qd7 pg6 nf6 pe6 na6 pd5 pb5 pa5 Pe4 Pg3 Be3 

# Results AxiomaticF2LOG

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.54, p@5: 0.5480000000000002, p@10: 0.524 <br> SCORE: @1: 54, @5: 60, @10: 40 | AVG: p@1: 0.63, p@5: 0.5559999999999999, p@10: 0.528 <br> SCORE: @1: 63, @5: 58, @10: 42 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.59, p@5: 0.5720000000000001, p@10: 0.5559999999999998 <br> SCORE: @1: 59, @5: 66, @10: 51 | AVG: p@1: 0.56, p@5: 0.5500000000000003, p@10: 0.5379999999999999 <br> SCORE: @1: 56, @5: 61, @10: 50 |

# Overview of results

__Results BM25__

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.55, p@5: 0.5540000000000002, p@10: 0.512 <br> SCORE: @1: 55, @5: 63, @10: 39         | AVG: p@1: 0.58, p@5: 0.5559999999999998, p@10: 0.5190000000000001 <br> SCORE: @1: 58, @5: 55, @10: 41         |
| Pre puzzle move (bad move)  | AVG: p@1: 0.51, p@5: 0.5700000000000001, p@10: 0.5440000000000002 <br> SCORE: @1: 51, @5: 61, @10: 48         |  AVG: p@1: 0.63, p@5: 0.5820000000000001, p@10: 0.5529999999999998 <br> SCORE: @1: 63, @5: 63, @10: 56        |

__Results TFIDF__

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.56, p@5: 0.5399999999999999, p@10: 0.4970000000000001 <br> SCORE: @1: 56, @5: 56, @10: 36 | AVG: p@1: 0.53, p@5: 0.5679999999999998, p@10: 0.529 <br> SCORE: @1: 53, @5: 60, @10: 44 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.51, p@5: 0.5459999999999999, p@10: 0.5619999999999998 <br> SCORE: @1: 51, @5: 52, @10: 47 | AVG: p@1: 0.58, p@5: 0.5960000000000001, p@10: 0.5559999999999999 <br> SCORE: @1: 58, @5: 67, @10: 54 |

__Results DFR__

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.56, p@5: 0.556, p@10: 0.526 <br> SCORE: @1: 56, @5: 62, @10: 41 | AVG: p@1: 0.57, p@5: 0.5419999999999998, p@10: 0.5220000000000001 <br> SCORE: @1: 57, @5: 54, @10: 41 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.52, p@5: 0.588, p@10: 0.552 <br> SCORE: @1: 52, @5: 66, @10: 51 | AVG: p@1: 0.64, p@5: 0.5660000000000001, p@10: 0.5519999999999999 <br> SCORE: @1: 64, @5: 58, @10: 55 |

__Results AxiomaticF2LOG__

|  / | Query Expansion | No Query Expansion |
|----------|----------|----------|
| Puzzle solution (good move) | AVG: p@1: 0.54, p@5: 0.5480000000000002, p@10: 0.524 <br> SCORE: @1: 54, @5: 60, @10: 40 | AVG: p@1: 0.63, p@5: 0.5559999999999999, p@10: 0.528 <br> SCORE: @1: 63, @5: 58, @10: 42 |
| Pre puzzle move (bad move)  | AVG: p@1: 0.59, p@5: 0.5720000000000001, p@10: 0.5559999999999998 <br> SCORE: @1: 59, @5: 66, @10: 51 | AVG: p@1: 0.56, p@5: 0.5500000000000003, p@10: 0.5379999999999999 <br> SCORE: @1: 56, @5: 61, @10: 50 |

# Investigate selected examples
...